In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
PATH = "thor-magni-actions/data/processed/thor_magni/"
SCENARIO_ID = "Scenario_3"

In [3]:
df = pd.read_csv(os.path.join(PATH, SCENARIO_ID + ".csv"), index_col="Time")

### Compute Path Efficiency

In [4]:
from scipy.spatial.distance import euclidean


def compute_path_effciency(trakclet):
    trakclet["cumsum_delta"] = trakclet["2D_norm_delta"].cumsum()
    first_location = (
        trakclet["x"].iloc[0],
        trakclet["y"].iloc[0],
    )
    trakclet["dist_origin_loc_i"] = trakclet.apply(
        lambda row: euclidean(first_location, (row["x"], row["y"])), axis=1
    )
    trakclet["path_efficiency"] = (
        trakclet["dist_origin_loc_i"] / trakclet["cumsum_delta"]
    )
    return trakclet

In [5]:
tracklets_grouped = df.groupby(
    ["file_name", "ag_id", "tracklet_id"], group_keys=False
).apply(compute_path_effciency)

In [6]:
tracklets_grouped = tracklets_grouped.groupby(["file_name", "ag_id", "tracklet_id"])
tracklets_grouped.ngroups

814

# Statistics

In [7]:
print("Agent types:", df["agent_type"].unique())
print("Actions:", df["action"].unique())

Agent types: ['Carrier-Box' 'Visitors-Group' 'Carrier-Bucket' 'Carrier-Large Object'
 'Visitors-Alone']
Actions: ['WalkBox' 'DeliverBox' 'Walk' 'PickBox' 'ObserveCardDraw' 'DrawCard'
 'PickBucket' 'WalkBucket' 'DeliverBucket' 'WalkLO']


In [8]:
global_stats = {
    "speed": [],
    "acceleration": [],
    "distance": [],
    "path_efficiency": [],
    "n_tracklets": 0,
}

specific_stats = {
    "agent_types": {
        at: {
            "speed": [],
            "acceleration": [],
            "distance": [],
            "path_efficiency": [],
            "n_tracklets": 0,
        }
        for at in df["agent_type"].unique()
    },
    "actions": {
        act: {"speed": [], "acceleration": [], "distance": [], "n_tracklets": 0}
        for act in df["action"].unique()
    },
}

for gn, participant_tracklet in tracklets_grouped:
    tracklets_ids = participant_tracklet["tracklet_id"].unique()
    for tracklet_id in tracklets_ids:
        tracklet_info = participant_tracklet[
            participant_tracklet["tracklet_id"] == tracklet_id
        ]
        global_stats["speed"].extend(tracklet_info["2D_speed"].values[1:])
        global_stats["acceleration"].extend(tracklet_info["2D_acc"].values[2:])
        global_stats["distance"].append(tracklet_info["2D_norm_delta"].sum())
        global_stats["path_efficiency"].append(tracklet_info["path_efficiency"].iloc[-1])
        global_stats["n_tracklets"] += 1

        agent_type = tracklet_info["agent_type"].iloc[0]
        specific_stats["agent_types"][agent_type]["speed"].extend(
            tracklet_info["2D_speed"].values[1:]
        )
        specific_stats["agent_types"][agent_type]["acceleration"].extend(
            tracklet_info["2D_acc"].values[2:]
        )
        specific_stats["agent_types"][agent_type]["distance"].append(
            tracklet_info["2D_norm_delta"].sum()
        )
        specific_stats["agent_types"][agent_type]["path_efficiency"].append(
            tracklet_info["path_efficiency"].iloc[-1]
        )
        specific_stats["agent_types"][agent_type]["n_tracklets"] += 1

        for curr_act in tracklet_info["action"].unique():
            target_info = tracklet_info[tracklet_info["action"] == curr_act]
            specific_stats["actions"][curr_act]["speed"].extend(
                target_info["2D_speed"].dropna()
            )
            specific_stats["actions"][curr_act]["acceleration"].extend(
                target_info["2D_acc"].dropna()
            )
            specific_stats["actions"][curr_act]["distance"].append(
                target_info["2D_norm_delta"].sum()
            )
            specific_stats["actions"][curr_act]["n_tracklets"] += 1

In [9]:
gl_speed = np.array(global_stats["speed"])
gl_acc = np.array(global_stats["acceleration"])
gl_dist = np.array(global_stats["distance"])
gl_peff = np.array(global_stats["path_efficiency"])
avg_global_speed, std_global_speed = gl_speed.mean(), gl_speed.std()
avg_global_acc, std_global_acc = gl_acc.mean(), gl_acc.std()
avg_global_dist, std_global_dist = gl_dist.mean(), gl_dist.std()
avg_global_peff, std_global_peff = gl_peff.mean(), gl_peff.std()

print("Global stats:")
print(f"Number of tracklets:{global_stats['n_tracklets']}")
print(f"Velocity:{avg_global_speed:1.2f}+-{std_global_speed:1.2f}")
print(f"Acceleration:{avg_global_acc:1.2f}+-{std_global_acc:1.2f}")
print(f"Distance:{avg_global_dist:1.2f}+-{std_global_dist:1.2f}")
print(f"Path Efficiency:{avg_global_peff:1.2f}+-{std_global_peff:1.2f}")

Global stats:
Number of tracklets:814
Velocity:0.96+-0.47
Acceleration:0.02+-0.38
Distance:7.58+-2.73
Path Efficiency:0.80+-0.23


In [10]:
agents_stats = specific_stats["agent_types"]
print("Stats per agent type:")

for agent, agent_stats in agents_stats.items():
    ag_speed = np.array(agent_stats["speed"])
    ag_acc = np.array(agent_stats["acceleration"])
    ag_dist = np.array(agent_stats["distance"])
    ag_peff = np.array(agent_stats["path_efficiency"])
    
    avg_ag_speed, std_ag_speed = ag_speed.mean(), ag_speed.std()
    avg_ag_acc, std_ag_acc = ag_acc.mean(), ag_acc.std()
    avg_ag_dist, std_ag_dist = ag_dist.mean(), ag_dist.std()
    avg_ag_peff, std_ag_peff = ag_peff.mean(), ag_peff.std()
    print(agent)
    print(f"Number of tracklets:{agent_stats['n_tracklets']}")
    print(f"Velocity:{avg_ag_speed:1.2f}+-{std_ag_speed:1.2f}")
    print(f"Acceleration:{avg_ag_acc:1.2f}+-{std_ag_acc:1.2f}")
    print(f"Distance:{avg_ag_dist:1.2f}+-{std_ag_dist:1.2f}")
    print(f"Path Efficiency:{avg_ag_peff:1.2f}+-{std_ag_peff:1.2f}")
    print()


Stats per agent type:
Carrier-Box
Number of tracklets:230
Velocity:1.15+-0.37
Acceleration:0.03+-0.41
Distance:9.07+-1.74
Path Efficiency:0.82+-0.23

Visitors-Group
Number of tracklets:129
Velocity:0.87+-0.44
Acceleration:0.02+-0.34
Distance:6.86+-2.27
Path Efficiency:0.81+-0.23

Carrier-Bucket
Number of tracklets:142
Velocity:1.22+-0.35
Acceleration:0.01+-0.46
Distance:9.71+-1.53
Path Efficiency:0.77+-0.26

Carrier-Large Object
Number of tracklets:285
Velocity:0.71+-0.45
Acceleration:0.02+-0.31
Distance:5.62+-2.60
Path Efficiency:0.80+-0.21

Visitors-Alone
Number of tracklets:28
Velocity:0.98+-0.55
Acceleration:0.01+-0.43
Distance:7.83+-2.08
Path Efficiency:0.84+-0.21



In [11]:
action_stats = specific_stats["actions"]
print("Stats per action:")

for action, act_stats in action_stats.items():
    act_speed = np.array(act_stats["speed"])
    act_acc = np.array(act_stats["acceleration"])
    act_dist = np.array(act_stats["distance"])
    avg_act_speed, std_act_speed = act_speed.mean(), act_speed.std()
    avg_act_acc, std_act_acc = act_acc.mean(), act_acc.std()
    avg_act_dist, std_act_dist = act_dist.mean(), act_dist.std()
    print(action)
    print(f"Number of tracklets:{act_stats['n_tracklets']}")
    print(f"Velocity:{avg_act_speed:1.2f}+-{std_act_speed:1.2f}")
    print(f"Acceleration:{avg_act_acc:1.2f}+-{std_act_acc:1.2f}")
    print(f"Distance:{avg_act_dist:1.2f}+-{std_act_dist:1.2f}")
    print()

Stats per action:
WalkBox
Number of tracklets:141
Velocity:1.20+-0.32
Acceleration:0.08+-0.33
Distance:6.80+-3.23

DeliverBox
Number of tracklets:78
Velocity:0.71+-0.43
Acceleration:-0.14+-0.64
Distance:1.97+-1.08

Walk
Number of tracklets:526
Velocity:1.14+-0.34
Acceleration:0.07+-0.32
Distance:6.19+-3.25

PickBox
Number of tracklets:57
Velocity:0.64+-0.42
Acceleration:-0.16+-0.62
Distance:1.26+-0.94

ObserveCardDraw
Number of tracklets:119
Velocity:0.29+-0.29
Acceleration:-0.08+-0.33
Distance:0.98+-0.80

DrawCard
Number of tracklets:150
Velocity:0.22+-0.27
Acceleration:-0.05+-0.38
Distance:0.78+-0.54

PickBucket
Number of tracklets:44
Velocity:0.78+-0.45
Acceleration:-0.25+-0.83
Distance:1.59+-0.86

WalkBucket
Number of tracklets:92
Velocity:1.28+-0.26
Acceleration:0.07+-0.32
Distance:6.59+-3.45

DeliverBucket
Number of tracklets:47
Velocity:0.82+-0.52
Acceleration:-0.24+-0.87
Distance:1.68+-0.83

WalkLO
Number of tracklets:158
Velocity:0.80+-0.35
Acceleration:0.01+-0.31
Distance:4.6